In [1]:
import pandas as pd
import uproot as ur
import numpy as np 
import matplotlib.pyplot as plt
import networkx as nx
import torch
import torch.nn as nn


from torch.utils.data import Dataset , DataLoader

In [2]:
# N_output -> number of inputs variables
N_layer1 = 25
N_layer2 = 25
N_layer3 = 25
N_layer4 = 25

do_dropout = True
Drop_rate = 0.2

In [30]:
file = pd.read_csv('./fracdata.csv',' ')

#df = file.arrays(library="pd")
df1 = file.head(70000)
#df1.columns
df1.keys()

/tmp/ipykernel_4312/3957282208.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  file = pd.read_csv('./fracdata.csv',' ')


Index(['seqNumber', 'runNumber', 'eventNumber', 'avgMu', 'nPrimVtx', 'jetCnt',
       'jetCalE', 'jetCalPt', 'jetCalEta', 'jetCalPhi', 'jetRawE', 'jetRawPt',
       'jetRawEta', 'jetRawPhi', 'jetNConst', 'truthJetMatchRadius',
       'truthJetE', 'truthJetPt', 'truthJetRap', 'truthJetPhi', 'nCluster',
       'clusterIndex', 'cluster_nCells', 'cluster_nCells_tot', 'clusterECalib',
       'clusterPtCalib', 'clusterEtaCalib', 'clusterPhiCalib',
       'cluster_sumCellECalib', 'cluster_fracECalib', 'cluster_fracECalib_ref',
       'clusterE', 'clusterPt', 'clusterEta', 'clusterPhi', 'cluster_sumCellE',
       'cluster_time', 'cluster_fracE', 'cluster_fracE_ref',
       'cluster_EM_PROBABILITY', 'cluster_HAD_WEIGHT', 'cluster_OOC_WEIGHT',
       'cluster_DM_WEIGHT', 'cluster_ENG_CALIB_TOT', 'cluster_ENG_CALIB_OUT_T',
       'cluster_ENG_CALIB_OUT_L', 'cluster_ENG_CALIB_OUT_M',
       'cluster_ENG_CALIB_DEAD_T', 'cluster_ENG_CALIB_DEAD_L',
       'cluster_ENG_CALIB_DEAD_M', 'cluster_ENG_CALI

In [69]:
#for event in eventNumbers:
tempE = []
tempEta = []
temp_time = []
temp_labels = []
count_jets = 1
old_event = df1['jetCnt'][0]
old_jetCnt = df1['jetCnt'][0]
change_bol = False

clusterE = []
clusterEta = []
cluster_time = []
labels = []

for index,row in df1.iterrows():
    
    clusterE.append(row['clusterE'])
    clusterEta.append(row['clusterEtaCalib'])
    cluster_time.append(row['cluster_time'])
    labels.append( int(row['cluster_ENG_CALIB_TOT'] <= 0) )


In [70]:
## create Dictionary containing data and labels

Dictionary_data ={
    "0": clusterE,
    "1": clusterEta,
    "2": cluster_time,
    "label": labels
}

In [71]:
len(Dictionary_data["2"])

70000

In [82]:
# Let's go to use an simple Neural Network :3 :3 :3
class NN_pileup(nn.Module):
    def __init__(self):
        super(NN_pileup,self).__init__()

        self.Drop = nn.Dropout( Drop_rate ) #0.25 debo testear esto hasta encontrar lo mejor!!!
        
        self.layer1 = nn.Linear(len(Dictionary_data)-1 ,N_layer1)   
        self.layer2 = nn.Linear(N_layer1,N_layer2) 
        self.layer3 = nn.Linear(N_layer2,N_layer3) 
        self.layer4 = nn.Linear(N_layer3,1) 
        #self.layer5 = nn.Linear(N_layer4,1)  
        
        self.activationReLU = nn.ReLU()
        self.activationLeaky = nn.LeakyReLU()
        self.activation = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)

    def  forward(self,x): #"pass the input through each of our operators"
        x = self.layer1(x)
        x = self.activationReLU(x)
        if do_dropout ==True:
            x = self.Drop(x)   ###!!!!!!!!!!!!
        
        x = self.layer2(x)
        x = self.activationReLU(x)
        if do_dropout ==True:
            x = self.Drop(x)   ###!!!!!!!!!!!!
        
        x = self.layer3(x)
        x = self.activationReLU(x)

        if do_dropout ==True:
            x = self.Drop(x)   ###!!!!!!!!!!!!
        
        x = self.layer4(x)
        '''
        x = self.activationReLU(x)
        if do_dropout ==True:
            x = self.Drop(x)   ###!!!!!!!!!!!!
        
        x = self.layer5(x)
        '''
        x = nn.functional.sigmoid(x)
        
        return x

In [83]:
class MyDataSet_train(torch.utils.data.Dataset):
    def __init__(self, Dictionary_data):
    
        #self.filename = filename
        self.EM14 = 1
        
    def __len__(self):
        return len(Dictionary_data["1"])
    
    def __getitem__(self, idx):

        features = []
        
        for i in range (0,len(Dictionary_data)-1):
            features.append(Dictionary_data[str(i)][idx])
            #print(len(features) )
            
        return np.array(features,dtype="float32" ), np.array( Dictionary_data["label"][idx],dtype="float32" )



In [84]:
Data_train = MyDataSet_train( Dictionary_data )

Batch_number = 1024*5

Data_train[26]

(array([ 1.6889977,  0.7223266, 16.72847  ], dtype=float32),
 array(0., dtype=float32))

In [88]:
#train_loader = DataLoader(Data_train,batch_size=Batch_number, shuffle=True,num_workers = 3)
train_loader = DataLoader(Data_train,batch_size=Batch_number, shuffle=True)

model = NN_pileup()

Adam_weight_decay = 0.002
learning_ratio = 0.005

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_ratio , weight_decay=Adam_weight_decay ) 
criterion = nn.CrossEntropyLoss()


#torch.nn.functional.binary_cross_entropy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion.to(device)

CrossEntropyLoss()

In [89]:
model.train()
epoch_loss=0
num_epochs= 40
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for data in train_loader:
        #data = data.to(device)
        #print(data[0])
        optimizer.zero_grad()
        out = model(data[0])
        out = out.view(-1, out.shape[-1])
        
        #labels = torch.tensor(data.y, dtype=torch.long).to(device) 
        labels = data[1]
        #print(out.shape)
        #print(labels.shape)
        
        labels = torch.reshape(labels, (int(list(labels.shape)[0]),1))
        #loss = criterion(out, labels)
        loss = torch.nn.functional.binary_cross_entropy(out, labels)
        #print(loss)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')



Epoch [1/40], Loss: 8.0410
Epoch [2/40], Loss: 5.9632
Epoch [3/40], Loss: 5.4223
Epoch [4/40], Loss: 5.2912
Epoch [5/40], Loss: 5.2181
Epoch [6/40], Loss: 5.1965
Epoch [7/40], Loss: 5.1679
Epoch [8/40], Loss: 5.1458
Epoch [9/40], Loss: 5.1306
Epoch [10/40], Loss: 5.1158
Epoch [11/40], Loss: 5.0996
Epoch [12/40], Loss: 5.0892
Epoch [13/40], Loss: 5.0818
Epoch [14/40], Loss: 5.0718
Epoch [15/40], Loss: 5.0499
Epoch [16/40], Loss: 5.0512
Epoch [17/40], Loss: 5.0538
Epoch [18/40], Loss: 5.0525
Epoch [19/40], Loss: 5.0351
Epoch [20/40], Loss: 5.0309
Epoch [21/40], Loss: 5.0413
Epoch [22/40], Loss: 5.0383
Epoch [23/40], Loss: 5.0351
Epoch [24/40], Loss: 5.0164
Epoch [25/40], Loss: 5.0269
Epoch [26/40], Loss: 5.0240
Epoch [27/40], Loss: 5.0195
Epoch [28/40], Loss: 5.0191
Epoch [29/40], Loss: 5.0200
Epoch [30/40], Loss: 5.0141
Epoch [31/40], Loss: 5.0105
Epoch [32/40], Loss: 4.9996
Epoch [33/40], Loss: 5.0027
Epoch [34/40], Loss: 5.0026
Epoch [35/40], Loss: 4.9840
Epoch [36/40], Loss: 5.0035
E

In [97]:
bb = np.array([1,2.2])
print(bb.dtype)
print( (bb.astype("float32")).dtype )
print(bb.dtype)


float64
float32
float64


In [100]:
float(False)

0.0